In [1]:
from unet_prax.src.networks import *

/Users/mromano/opt/anaconda3/envs/unet_prax-env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(24*16)
])

TRAIN = MNIST(root='.', train=True, transform=transform)
TEST = MNIST(root='.', train=False, transform=transform)

class MnistData(Dataset):
    def __init__(self, train: bool):
        super().__init__()
        if train:
            self._data = TRAIN
        else:
            self._data = TEST
    
    def _binaryitem(self, idx):
        d = self._data[idx][0]
        return torch.where(d == 0, 0, 1)

    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):
        x = self._data[idx][0]
        y = self._binaryitem(idx)
        return x, y


In [4]:
n_epochs = 100

net = UNet(1, channel_list=(1,12,24,48,96,192,))

loss_fun = nn.BCEWithLogitsLoss()
optim = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
optim.zero_grad()

train_data = MnistData(True)
test_data = MnistData(False)

train_dataloader = DataLoader(train_data, batch_size=100, drop_last=True)
test_dataloader = DataLoader(test_data, batch_size=len(test_data))



        

In [5]:
test_data[0][0].shape

torch.Size([1, 384, 384])

In [7]:

for epoch in tqdm(range(n_epochs)):
    for samples, labels in train_dataloader:
        optim.zero_grad()
        out = net(samples)
        print(out.shape)
        loss = loss_fun(out, samples)
        print(loss)
        loss.back()

  0%|          | 0/100 [00:04<?, ?it/s]

torch.Size([100, 2, 196, 196])


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

###### Initialize network weights as follows :  "For a network with our architecture (alternating convolution and ReLU layers) this can be achievedby  drawing  the  initial  weights  from  a  Gaussian  distribution  with  a  standard deviation of √2/N, where N denotes the number of incoming nodes of one neuron [5]. E.g. for a 3x3 convolution and 64 feature channels in the previous layer N= 9·64 = 576"


In [ ]:
'''
only convolutional and upsampleolutional layers have learnable parameters.
So, search for each convolutional layer, to initialize, 
simply ask what the previous convolutional layer's kernel size was,
and find the number of channels of that previous layer (dim[1])
'''

"\nonly convolutional and upsampleolutional layers have learnable parameters.\nSo, search for each convolutional layer, to initialize, \nsimply ask what the previous convolutional layer's kernel size was,\nand find the number of channels of that previous layer (dim[1])\n"